In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
mistral_api_key = os.environ["MISTRAL_API_KEY"]

In [5]:
from langchain_mistralai import ChatMistralAI

chatModel = ChatMistralAI(
    mistral_api_key=mistral_api_key,
    model="mistral-large-latest",
    temperature=0.7,
    max_retries=2,
    # other params...
)

In [6]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader = WebBaseLoader(
    web_paths=("https://vaastuvidwan.com/how-to-use-pyramid-for-vastu-correction/",
              "https://www.houseofhiranandani.com/vlogs/all-you-need-to-know-about-vastu-before-buying-a-new-home/",
              "https://www.niranjanbabu.com/vastu-for-homes.html", 
              "https://www.fortiusinfra.com/buying-a-vastu-compliant-apartment-in-bangalore/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("elementor elementor-3113 elementor-location-single post-3025 post type-post status-publish format-standard has-post-thumbnail hentry category-uncategorized tag-pyramid-vastu",
                   "entry-content", "container", "inner-content", )
        )
    ),
)

In [8]:
docs = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)

In [10]:
splits = text_splitter.split_documents(docs)

In [11]:
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

/Users/pramodmodi/Library/Caches/pypoetry/virtualenvs/langchain-mistral-xuCIEomg-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## Add pdf data

In [12]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('./data/vastu-for-house.pdf')

pdf_loaded_data = pdf_loader.load_and_split()

In [14]:
vectorstore.add_documents(documents=pdf_loaded_data)

['197feec1-c6f8-4252-8f5b-9f55402636b5',
 '4e9ccd9a-5b44-4fb4-b883-8373553e96cc',
 '60745eed-20a8-4e67-a565-d540c1e1bd5a',
 '670d2bf0-3a74-4882-8ef7-e0555e786f88',
 '0357ebd4-5865-479f-9864-a4616b22bc36',
 '16a9063e-52e7-4dca-8b66-7141c51081e7',
 'a54b24a6-fadb-4d1d-be69-764ab381777f',
 '076f8899-08be-4348-b52f-ba5839fec8d7',
 '2bb69331-ac20-41f0-86c5-2e41968e44ae',
 '8f041ce0-49c7-41d4-935e-8b2a8920c5ad',
 'e4c2379a-9479-4d33-ac9e-30d19885ef6c',
 '5d0005d6-0119-4a17-99b1-333ccea31cc5',
 '26ef4cd8-082e-44b7-8688-9038c0e8a8a9',
 '51e11a24-4e79-4bc6-b686-5a2532562b82',
 '36675a57-acbf-492a-875d-6343b2c9c10d',
 '90fd4fc9-9258-4cb2-b4d7-cd88b02c7cb0',
 '538f7e0c-cee3-4f09-9b12-532eab5c33ec',
 'c8d08cd8-79f0-421b-8d95-ea484eca2174',
 'de4f913c-e98f-4384-aba2-2d9ea6012e7b',
 '82d83c26-659a-4eee-82c6-4cb7bbca6f9d',
 '7af62b5f-1b77-4e4e-8dc8-85bd1df1a222',
 'fb6b365c-2da6-4db1-ae7c-f48398097877',
 '444a6c31-b9fc-45e9-84e4-ea73067d1a2f',
 'ad66c429-149d-4a34-8841-8b604e43041f',
 '67651f35-a4cd-

In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

#prompt = hub.pull("rlm/rag-prompt")

In [21]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context. If context is not available then you reply based on your information:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [22]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [23]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)

In [24]:
query= "According to Vastu where should be kitchen situated?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

According to Vastu principles, the kitchen should be situated in the South-East corner of the house. It is also important to ensure that the kitchen is not directly facing the main door of the house.

In [25]:
query= "If kitechen is not according to Vastu, then what is alternatives"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Based on the provided context, if the kitchen is not positioned according to Vastu principles, here are some alternatives and corrections you can consider:

1. **Use of Colors**: Paint the kitchen walls with colors that represent the fire element, such as red, orange, or saffron, to strengthen the element's presence.

2. **Placement of Stove**: If possible, relocate the stove so that it is in the southeast corner of the kitchen. If this is not feasible, ensure that the person cooking faces east while cooking.

3. **Mirror Reflection**: Place a mirror on the wall opposite the stove so that it reflects the burners. This is believed to create a virtual southeast direction.

4. **Use of Crystals**: Hang a pyramid or sphere made of clear quartz crystal above the stove to help balance the energies.

5. **Lighting**: Ensure the kitchen is well-lit, as light is considered a good substitute for the fire element.

6. **Plants**: Place a living green plant in the kitchen to invite positive energy

In [26]:
query= "House having west facing entrance, with south east corner kitchen and master bed room in north. Is it accordance with Vastu. If not then what remedies are needed?"
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

Based on the provided context about Vastu compliance, the house you described has the following issues:

1. **West-facing entrance**: This is not considered ideal as the best entries are east or north-east, followed by north-west. West is not mentioned as a favorable direction for the main entrance.

2. **South-east corner kitchen**: This is in accordance with Vastu principles as the south-east corner is considered the ideal spot for the kitchen.

3. **Master bedroom in the north**: This is not in accordance with Vastu principles, which suggest that the master bedroom should be in the south-west corner of the house.

Here are some remedies to help mitigate these issues:

1. **For the west-facing entrance**:
   - Place a mirror on the wall opposite the main door to deflect any negative energy.
   - Keep the entrance well-lit and clean to attract positive energy.
   - You can also place a plant near the entrance, but ensure it's not a thorny or cactus plant.

2. **For the master bedroom 